## Update Bioconductor

In [ ]:
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install(version = "3.17")

## Install Packages

In [65]:
# ComplexHeatmap
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install("ComplexHeatmap")

# dplyr
install.packages("dplyr")

# tidy4
install.packages("tidyr")

# reshape2
install.packages("reshape2")

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.r-project.org

Bioconductor version 3.16 (BiocManager 1.30.21), R 4.2.0 (2022-04-22)

Warning message:
"package(s) not installed when version(s) same as or greater than current; use
  `force = TRUE` to re-install: 'ComplexHeatmap'"
Old packages: 'aplot', 'askpass', 'BiocManager', 'bslib', 'Cairo', 'CMplot',
  'cpp11', 'credentials', 'curl', 'dqrng', 'DT', 'emmeans', 'fastmatch',
  'fontawesome', 'foreign', 'fs', 'gert', 'ggfun', 'ggplot2', 'ggplotify',
  'gtable', 'htmltools', 'httr', 'igraph', 'knitr', 'labeling', 'mapplots',
  'markdown', 'Matrix', 'mvtnorm', 'nlme', 'openssl', 'patchwork', 'progressr',
  'promises', 'purrr', 'RcppAnnoy', 'RcppArmadillo', 'rematch', 'remotes',
  'reticulate', 'rmarkdown', 'sass', 'scCustomize', 'shiny', 'snakecase',
  'spatial', 'spatstat.explore', 'spatstat.geom', 'spatsta

## Load Packages

In [4]:
library(ComplexHeatmap)
library(dplyr)
library(tidyr)
library(reshape2)

Loading required package: grid

ComplexHeatmap version 2.14.0
Bioconductor page: http://bioconductor.org/packages/ComplexHeatmap/
Github page: https://github.com/jokergoo/ComplexHeatmap
Documentation: http://jokergoo.github.io/ComplexHeatmap-reference

If you use it in published research, please cite either one:
- Gu, Z. Complex Heatmap Visualization. iMeta 2022.
- Gu, Z. Complex heatmaps reveal patterns and correlations in multidimensional 
    genomic data. Bioinformatics 2016.


The new InteractiveComplexHeatmap package can directly export static 
complex heatmaps into an interactive Shiny app with zero effort. Have a try!

This message can be suppressed by:
  suppressPackageStartupMessages(library(ComplexHeatmap))



Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




## Load Dataframe

In [42]:
df <- read.csv("tables_and_figures/expected_vs_observed_peaks_master.csv", header = TRUE)

## Create Heatmap Matrices

In [51]:
# Create heatmap matrices

# Select data of interest
heatmap_data <- df %>%
  select(Endedness, Peak_Type, Aligner, Peak_Caller, Deduplicator, Test_Dataset, Control, Sensitivity, Precision, F1_Score)

# Combine variables for plotting
heatmap_data$Seq_Metadata <- paste(heatmap_data$Endedness, heatmap_data$Peak_Type, sep = "_")

# Replace NA values with 0
heatmap_data <- heatmap_data %>%
  mutate(Sensitivity = replace_na(Sensitivity, 0),
         Precision = replace_na(Precision, 0),
         F1_Score = replace_na(F1_Score, 0))

mat <- dcast(heatmap_data, Test_Dataset ~ Aligner, value.var = "F1_Score", fun.aggregate = sum)

# Set row names to Test_Dataset numbers
rownames(mat) <- mat$Test_Dataset

# Remove the Test_Dataset column
mat <- mat[, -1]

# Print the resulting matrix
print(mat)

   bowtie2  bwa_mem     STAR
1 30.91541 30.91541 30.91541
2 43.55069 43.55069 43.55069
3 48.65071 48.65071 48.65071
4 52.50032 52.50032 52.50032
5 49.55741 49.55741 49.55741
6 51.48506 51.48506 51.48506


## Generate Heatmaps

In [43]:
#pdf("F1_score_heatmap.pdf", height = 20, width = 10)

row_ha = rowAnnotation(F1_Scores = rownames(male_matrix))
column_ha = HeatmapAnnotation(`Peak Caller` = heatmap_matrix$Peak_Caller, 
                              `Deduplicator` = heatmap_matrix$Deduplicator,
                              `Test Dataset` = heatmap_matrix$Test_Dataset,
                              col = list(`Peak Caller` = c("cisgenome" = "#c2ffc3","genrich"= "#90ec7c","macs3"= "#00b22a", "pepr" = "#006b05"), 
                                         `Deduplicator` = c("samtools" = "#e6e6fa", "no_deduplication" = "#aea3e0", "picard" = "#835fbd", "sambamba" = "#620093"),
                                         `Test Dataset` = c("1" = "#00CCFF", "2" = "#00A3FF", "3" = "#007AFF", "4" = "#0052FF", "5" = "#0000FF", "6" = "#0700C4", ), 
                                         annotation_name_gp = gpar(fontsize = 16 )),
                              annotation_name_gp = gpar(fontsize = 16 ))

map = grid.grabExpr(
  draw(
    Heatmap(heatmap_matrix, 
            name = "F1 Scores", 
            top_annotation = column_ha, 
            col = col_fun, 
            row_names_gp=gpar(fontsize=16,  ),
            column_names_gp=gpar(fontsize=0),
            cluster_columns = FALSE, 
            heatmap_legend_param = list(title="F1 Scores", 
                                        title_gp = gpar(fontsize = 16,  ), 
                                        labels_gp = gpar(fontsize = 12,  )), 
)))

#grid.newpage()
grid.draw(map)

#dev.off()

ERROR: Error in rownames(male_matrix): object 'male_matrix' not found
